In [1]:
%cd ..

/workspaces/pythainlp


In [2]:
from pythainlp.transliterate.thai2rom import ThaiTransliterator

_THAI_TO_ROM = ThaiTransliterator()

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pythainlp.transliterate import romanize

print(romanize('ทดสอบ', engine = 'thai2rom'))
print(romanize('ทดสอบ', engine = 'thai2rom_onnx'))

thotsop
thotsop


In [5]:
import torch
from pythainlp.corpus import get_corpus_path
from pythainlp.transliterate.thai2rom import _MODEL_NAME

loader = torch.load(get_corpus_path(_MODEL_NAME), "cpu")

In [6]:
loader_onnx = {
    "output_dim": loader["decoder_params"][0],
    "ix_to_char": loader["ix_to_char"],
    "char_to_ix": loader["char_to_ix"],
    "target_char_to_ix": loader["target_char_to_ix"],
    "ix_to_target_char": loader["ix_to_target_char"],
}

In [7]:
import json

with open("thai2rom_loader_onnx.json", "w") as f:
    json.dump(loader_onnx, f)

In [8]:
import torch
import numpy as np

input_tensor = torch.Tensor([[30, 19, 8, 30, 38, 37, 10, 3]]).long()

input_length = torch.Tensor([8]).int()

dummy_vars_encoder = (input_tensor, input_length)

In [9]:
decoder_input = torch.Tensor([[9]]).int()
decoder_hidden = torch.rand(1, 1, 256)
encoder_outputs = torch.rand(1, 11, 256)
mask = torch.Tensor(
    [[True, True, True, True, True, True, True, True, True, True, True]]
)

dummy_vars_decoder = (decoder_input, decoder_hidden, encoder_outputs, mask)

In [10]:
result_original_decoder = _THAI_TO_ROM._network.decoder(*dummy_vars_decoder)

In [11]:
result_original_encoder = _THAI_TO_ROM._network.encoder(*dummy_vars_encoder)

In [12]:
dynamic_axes = {
    "input_tensor": {1: "input_length"},
    "encoder_outputs": {1: "encoder_output_length"},
}

torch.onnx.export(
    _THAI_TO_ROM._network.encoder,
    dummy_vars_encoder,
    "thai2rom_encoder.onnx",  # verbose=True, \
    input_names=["input_tensor", "input_lengths"],
    output_names=["encoder_outputs", "encoder_hidden_1", "encoder_hidden_2"],
    opset_version=13,
    dynamic_axes=dynamic_axes,
)

/usr/local/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:1941: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn("Exporting a model to ONNX with a batch_size other than 1, " +


In [13]:
decoder_input = torch.Tensor([[9]]).int()
decoder_hidden = torch.rand(1, 1, 256)
encoder_outputs = torch.rand(1, 11, 256)
mask = torch.Tensor(
    [[True, True, True, True, True, True, True, True, True, True, True]]
)

dummy_vars_decoder = (decoder_input, decoder_hidden, encoder_outputs, mask)

In [14]:
dynamic_axes = {"encoder_outputs": {1: "input_length"}, "mask": {1: "input_length"}}

torch.onnx.export(
    _THAI_TO_ROM._network.decoder,
    dummy_vars_decoder,
    "thai2rom_decoder.onnx",  # verbose=True, \
    input_names=["decoder_input", "decoder_hidden_1", "encoder_outputs", "mask"],
    output_names=["decoder_output", "decoder_hidden_2"],
    opset_version=13,
    dynamic_axes=dynamic_axes,
)

In [15]:
from onnxruntime import InferenceSession

encoder_session = InferenceSession("./thai2rom_encoder.onnx")
decoder_session = InferenceSession("./thai2rom_decoder.onnx")